In [1]:
import os


In [4]:
%pwd

'c:\\Users\\heman\\Desktop\\Text-Summarizer'

In [3]:
os.chdir("../")

In [5]:
from pathlib import Path
from dataclasses import dataclass

In [6]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path : Path
    model_name : Path

In [7]:
from src.Text_Summarizer.constants import *
from src.Text_Summarizer.utils.common import * 

In [8]:
class ConfigurationManagerMT:
    def __init__(self , config= CONFIG_YAML_PATH , params = PARAM_YAML_PATH):
        self.config = read_yaml(config) 
        self.params = read_yaml(params)
        
        
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        get_model_trainer = self.config.model_trainer
        create_directories([get_model_trainer.root_dir])
        get_model_trainer_config = ModelTrainerConfig(
            root_dir=get_model_trainer.root_dir,
            data_path=get_model_trainer.data_path,
            model_name=get_model_trainer.model_name
            
        )
        
        return get_model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer , AutoModelForSeq2SeqLM , AutoTokenizer,DataCollatorForSeq2Seq
from src.Text_Summarizer.logging import logger
from datasets import load_from_disk
import torch


c:\Users\heman\Desktop\Text-Summarizer\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-01-18 18:10:31,469 : INFO : config : PyTorch version 2.5.1 available.]


In [14]:
class ModelTrainer:
    def __init__(self , config : ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)  #load a tokenizer

        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_name)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
        per_device_train_batch_size=1, per_device_eval_batch_size=1,
        weight_decay=0.01, logging_steps=10,
        evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
        gradient_accumulation_steps=16
        )
        
        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])
        
        
        trainer.train()
        
        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"Trained_pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"trained_tokenizer"))
        
        
        
        
        

In [20]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate
# !pip install transformers[torch]

In [25]:
# pip install accelerate


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/73/de/64508cb91af013aaba214752309c0967568a4219d50a4ea30e822af3c976/accelerate-1.3.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/336.6 kB ? eta -:--:--
   -- ------------------------------------ 20.5/336.6 kB 640.0 kB/s eta 0:00:01
   ------------------------ --------------- 204.8/336.6 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 336.6/336.6 kB 3.5 MB/s eta 0:00:00


In [15]:
config = ConfigurationManagerMT()
model_Trainer_Config = config.get_model_trainer_config()
model_train = ModelTrainer(config=model_Trainer_Config)
model_train.train()

[2025-01-18 18:13:01,315 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-01-18 18:13:01,315 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-01-18 18:13:01,317 : INFO : common : created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\heman\AppData\Local\Temp\ipykernel_8664\1524608223.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args=trainer_args,


Step,Training Loss,Validation Loss


KeyboardInterrupt: 